<a href="https://colab.research.google.com/github/AnkitaKumariJain14/Malaria-Detection-Using-ML/blob/master/Malaria_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Malaria/cell_images.zip
!unzip cell_images.zip
!rm cell_images/Uninfected/Thumbs.db
!rm cell_images/Parasitized/Thumbs.db

In [12]:
#Importing Necessary Libraries.
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

In [3]:

data=[]
labels=[]
Parasitized=os.listdir('/content/cell_images/Parasitized/') 
for a in Parasitized:
    try:
        image=cv2.imread("/content/cell_images/Parasitized/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((64, 64))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")

Uninfected= os.listdir('/content/cell_images/Uninfected/')
for b in Uninfected:
    try:
        image=cv2.imread("/content/cell_images/Uninfected/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((64, 64))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

In [4]:
Cells=np.array(data)
labels=np.array(labels)
s=np.arange(Cells.shape[0])
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]
num_classes=len(np.unique(labels))
len_data=len(Cells)

In [5]:
(x_train,x_test)=Cells[(int)(0.1*len_data):],Cells[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 # As we are working on image data we are normalizing data by divinding 255.
x_test = x_test.astype('float32')/255
train_len=len(x_train)
test_len=len(x_test)

In [6]:
(y_train,y_test)=labels[(int)(0.1*len_data):],labels[:(int)(0.1*len_data)]
#Doing One hot encoding as classifier has multiple classes
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [7]:
model=Sequential()
model.add(Conv2D(filters=6,kernel_size=2,padding="same",activation="relu",input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Flatten())
model.add(Dense(120,activation="relu"))
model.add(Dense(84,activation="relu"))
model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 6)         78        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 16)        400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0         
_________________________________________________________________
dense (Dense)                (None, 120)               491640    
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [8]:
# compile the model with loss as categorical_crossentropy and using adam optimizer you can test result by trying RMSProp as well as Momentum
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
#Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
model.fit(x_train,y_train,batch_size=50,epochs=20,verbose=1)

Epoch 1/20
497/497 [==============================] - 42s 85ms/step - loss: 0.5357 - accuracy: 0.7164
Epoch 2/20
497/497 [==============================] - 42s 85ms/step - loss: 0.2437 - accuracy: 0.9054
Epoch 3/20
497/497 [==============================] - 42s 84ms/step - loss: 0.1784 - accuracy: 0.9370
Epoch 4/20
497/497 [==============================] - 42s 84ms/step - loss: 0.1439 - accuracy: 0.9509
Epoch 5/20
497/497 [==============================] - 42s 85ms/step - loss: 0.1183 - accuracy: 0.9576
Epoch 6/20
497/497 [==============================] - 42s 85ms/step - loss: 0.0948 - accuracy: 0.9679
Epoch 7/20
497/497 [==============================] - 42s 85ms/step - loss: 0.0793 - accuracy: 0.9721
Epoch 8/20
497/497 [==============================] - 46s 93ms/step - loss: 0.0635 - accuracy: 0.9777
Epoch 9/20
497/497 [==============================] - 43s 86ms/step - loss: 0.0455 - accuracy: 0.9848
Epoch 10/20
497/497 [==============================] - 42s 85ms/step - loss: 0.035

In [10]:
accuracy = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test_Accuracy:-', accuracy[1])

87/87 [==============================] - 3s 29ms/step - loss: 0.4098 - accuracy: 0.9376

 Test_Accuracy:- 0.9375680685043335


In [11]:
from keras.models import load_model
model.save('cells.h5')